# Introduction To Astro-Data Analysis

In this notebook, I will cover the key methods required to get started with processing astrophysical datasets.

We will cover:
- Downloading datasets directly from the source using scripts
- Dealing with different file formats
- Loading in data effectively
- Processing and manipulating data
- Visulation
- Storing Data

## Case Study
Gamma-Ray Bursts (GRBs) are among the highest energy events that arise within the Universe. They can outshine galaxies and can be seen at extremely high redshift. There are two primary classes of GRBs, short and long. These are characterised by the time taken for 5\% - 95\% of the flux to be detected (T90). Short GRBs have T90s typically <2 s, whilst for Long GRBs, the T90s are >2 s.  

In this example, we will:
- Download some GRB light curves
- Visualise
- Download and visualise location skymaps corresponding to the GRB
- Identify the nearest galaxies
- Circle regions of interest
- Output a list of nearby galaxies

## Downloading data


In [34]:
"""Conveniently, python has native packages that can help us download data directly from HTTP sources! 
The [urlretrieve] package is what we will be using to download the dataset that we are interested in."""
import urllib.request as req
import shutil #for shell operations
"Download the data for GRB 211211A - Full list can be found here: https://www.swift.ac.uk/xrt_products/index.php?year=2022"
"Click on the curve option for the GRB of interest and click the download button. You can either save the file directly or copy the url."

"Direct Download"
url  = "https://www.swift.ac.uk/scripts/viewData.php?file=https://www.swift.ac.uk/user_objects/tprods/tmp_jsplot_bqmZlb.qdp"
text_file = 'GRB211211A_xrt.txt'
try:
    data = req.urlretrieve(url,text_file) #Legacy method
except:
    print("Legacy method has failed")
    "Modern approach to downloading datasets"
    with req.urlopen(url) as response, open(text_file,'wb') as of:
        shutil.copyfileobj(response,of)

Legacy method has failed


## Loading in dataset
Now that we have downloaded our data, we need to load it in, so we can start processing it! 

In [30]:
import pandas as pd
import numpy as np
import csv


,,,,,! WTSLEWHRhard
70.9036887166384,1.515502,-1.557131,1049.918423,67.731397,-67.731397
75.651008093708,3.104449,-3.231817,795.918974,51.304162,-51.304162
NO NO NO NO NO NO,NaN,NaN,NaN,NaN,NaN
! WTHRhard,NaN,NaN,NaN,NaN,NaN
84.015,1.227000,-1.041000,776.987414,57.487909,-57.487909
...,...,...,...,...,...
17183.751,246.438000,-242.489000,0.819197,0.180001,-0.180001
21907.294,238.860000,-237.527000,0.756893,0.171413,-0.171413
58973.289,3872.893000,-2710.837000,0.997328,0.270894,-0.270894
67907.252,482.935000,-612.755000,1.249625,0.437256,-0.437256


### Using Pandas


In [ ]:
"The dataset has 4 headers but 6 columns"
pd.read_csv(text_file,sep='\t',skiprows=[0])